<a href="https://colab.research.google.com/github/dibyanshu12/Topic-Modeling/blob/main/HDP_Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pandas import option_context
import numpy as np
import string
import re
import sys

# Sklearn
from sklearn.datasets import fetch_20newsgroups

# Plotting
import matplotlib.pyplot as plt

In [ ]:
import nltk
nltk.download('stopwords', quiet=True)

from nltk.corpus import stopwords
st_words = stopwords.words('english')
st_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
news = fetch_20newsgroups(subset='train') # import fron skleanr

In [ ]:
# Build master df

news_df = pd.DataFrame([news.target, news.data]).T
news_df = news_df.set_index(0)
#news_df = pd.concat([news_df, pd.Series(news.target_names)],axis=1, join="inner")
news_df.reset_index(inplace=True)
news_df.columns = ["topic_id", "content"]

news_df.head()

,topic_id,content
0,7,From: lerxst@wam.umd.edu (where's my thing)\nS...
1,4,From: guykuo@carson.u.washington.edu (Guy Kuo)...
2,4,From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3,1,From: jgreen@amber (Joe Green)\nSubject: Re: W...
4,14,From: jcm@head-cfa.harvard.edu (Jonathan McDow...


In [ ]:
doc_list = news_df.content.values.tolist()

In [ ]:
doc_list[2]

'From: twillis@ec.ecn.purdue.edu (Thomas E Willis)\nSubject: PB questions...\nOrganization: Purdue University Engineering Computer Network\nDistribution: usa\nLines: 36\n\nwell folks, my mac plus finally gave up the ghost this weekend after\nstarting life as a 512k way back in 1985.  sooo, i\'m in the market for a\nnew machine a bit sooner than i intended to be...\n\ni\'m looking into picking up a powerbook 160 or maybe 180 and have a bunch\nof questions that (hopefully) somebody can answer:\n\n* does anybody know any dirt on when the next round of powerbook\nintroductions are expected?  i\'d heard the 185c was supposed to make an\nappearence "this summer" but haven\'t heard anymore on it - and since i\ndon\'t have access to macleak, i was wondering if anybody out there had\nmore info...\n\n* has anybody heard rumors about price drops to the powerbook line like the\nones the duo\'s just went through recently?\n\n* what\'s the impression of the display on the 180?  i could probably swin

In [ ]:
# Remove Emails
doc_list = [re.sub(r'\S*@\S*\s?', '', sent) for sent in doc_list]

# Remove new line characters
doc_list = [re.sub(r'\s+', ' ', sent) for sent in doc_list]

# Remove distracting single quotes
doc_list = [re.sub(r"\'", "", sent) for sent in doc_list]

In [ ]:
doc_list[2]

'From: (Thomas E Willis) Subject: PB questions... Organization: Purdue University Engineering Computer Network Distribution: usa Lines: 36 well folks, my mac plus finally gave up the ghost this weekend after starting life as a 512k way back in 1985. sooo, im in the market for a new machine a bit sooner than i intended to be... im looking into picking up a powerbook 160 or maybe 180 and have a bunch of questions that (hopefully) somebody can answer: * does anybody know any dirt on when the next round of powerbook introductions are expected? id heard the 185c was supposed to make an appearence "this summer" but havent heard anymore on it - and since i dont have access to macleak, i was wondering if anybody out there had more info... * has anybody heard rumors about price drops to the powerbook line like the ones the duos just went through recently? * whats the impression of the display on the 180? i could probably swing a 180 if i got the 80Mb disk rather than the 120, but i dont really 

In [ ]:
# Gensim
import gensim

word_list = [gensim.utils.simple_preprocess(txt, deacc=True, min_len=3) for txt in doc_list]

In [ ]:
word_list[2][:6]

['from', 'thomas', 'willis', 'subject', 'questions', 'organization']

In [ ]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser

bigram = Phrases(word_list, min_count=5, threshold=100)

bigram_model = Phraser(bigram)

In [ ]:
# Example of stop word removal process
sentence = "A letter has been written asking him to be released"
for word in sentence.split():
    if word not in st_words:
        print("Kept ==> " +word)
    else:
        print("Removed: " +word)

Kept ==> A
Kept ==> letter
Removed: has
Removed: been
Kept ==> written
Kept ==> asking
Removed: him
Removed: to
Removed: be
Kept ==> released


In [ ]:
word_list_nostops = [[word for word in txt if word not in st_words] for txt in word_list]
word_list_nostops[2][:6]

['thomas', 'willis', 'questions', 'organization', 'purdue', 'university']

In [ ]:
word_bigrams = [bigram_model[w_vec] for w_vec in word_list_nostops]

In [ ]:
word_bigrams[2][:7]

In [ ]:
import spacy

## Example showing lemmatization
sentence = "A letter has been written asking him to be released"

spC = spacy.load('en_core_web_sm')

lemma_sentence = spC(sentence)
for token in lemma_sentence:
    print(token.text + " ==> " +token.lemma_ + ", " + token.pos_)

A ==> a, DET
letter ==> letter, NOUN
has ==> have, AUX
been ==> be, AUX
written ==> write, VERB
asking ==> ask, VERB
him ==> -PRON-, PRON
to ==> to, PART
be ==> be, AUX
released ==> release, VERB


In [ ]:
spC = spacy.load('en_core_web_sm')
postags = ['NOUN','VERB', 'ADV', 'ADJ'] # Keep nouns, adj, verbs, and adverbs


def lemmatize(word_list, ptags = postags ):
    '''Lemmatizes words based on allowed postags, input format is list of sublists
       with strings'''

    lem_lists =[]
    for vec in word_list:
        sentence = spC(" ".join(vec))
        lem_lists.append([token.lemma_ for token in sentence if token.pos_ in ptags])

    return lem_lists

In [ ]:
# Lemmatize bigrams
word_list_lemmatized = lemmatize(word_bigrams)

In [ ]:
word_list_lemmatized[2][:7]

['engineering',
 'computer',
 'network',
 'distribution_usa',
 'line',
 'well',
 'folk']

In [ ]:
!pip install tomotopy

     |████████████████████████████████| 16.3 MB 6.8 MB/s 


In [ ]:
import tomotopy as tp

hdp = tp.HDPModel(tw=tp.TermWeight.IDF, min_cf=5, rm_top=7,
                 gamma=1, alpha=0.1, initial_k=10, seed=99999)

# Add docs to train
for vec in word_list_lemmatized:
    hdp.add_doc(vec)

# Initiate MCMC burn-in
hdp.burn_in = 100
hdp.train(0)
print('Num docs:', len(hdp.docs), ', Vocab size:', hdp.num_vocabs, ', Num words:', hdp.num_words)
print('Removed top words:', hdp.removed_top_words)
print('Training...', file=sys.stderr, flush=True)

Training...


Num docs: 11314 , Vocab size: 12716 , Num words: 1004667
Removed top words: ['line', 'would', 'write', 'say', 'know', 'article', 'people']


In [ ]:
# Train model
mcmc_iter=1000
for i in range(0, mcmc_iter, 100):
    hdp.train(100, workers=3)
    print('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, hdp.ll_per_word, hdp.live_k))

Iteration: 0	Log-likelihood: -8.040162289508805	Num. of topics: 18
Iteration: 100	Log-likelihood: -8.036344954011918	Num. of topics: 19
Iteration: 200	Log-likelihood: -8.036610465558239	Num. of topics: 21
Iteration: 300	Log-likelihood: -8.03494260430024	Num. of topics: 21
Iteration: 400	Log-likelihood: -8.035102852242396	Num. of topics: 19
Iteration: 500	Log-likelihood: -8.034813546651522	Num. of topics: 18
Iteration: 600	Log-likelihood: -8.035080041624965	Num. of topics: 19
Iteration: 700	Log-likelihood: -8.03520256431882	Num. of topics: 18
Iteration: 800	Log-likelihood: -8.03547803833439	Num. of topics: 19
Iteration: 900	Log-likelihood: -8.035228853412798	Num. of topics: 18


In [ ]:
def get_hdp_topics(hdp, top_n=10):
    '''Wrapper function to extract topics from trained tomotopy HDP model

    ** Inputs **
    hdp:obj -> HDPModel trained model
    top_n: int -> top n words in topic based on frequencies

    ** Returns **
    topics: dict -> per topic, an arrays with top words and associated frequencies
    '''

    # Get most important topics by # of times they were assigned (i.e. counts)
    sorted_topics = [k for k, v in sorted(enumerate(hdp.get_count_by_topics()), key=lambda x:x[1], reverse=True)]

    topics=dict()

    # For topics found, extract only those that are still assigned
    for k in sorted_topics:
        if not hdp.is_live_topic(k): continue # remove un-assigned topics at the end (i.e. not alive)
        topic_wp =[]
        for word, prob in hdp.get_topic_words(k, top_n=top_n):
            topic_wp.append((word, prob))

        topics[k] = topic_wp # store topic word/frequency array

    return topics

In [ ]:
hdp_topics = get_hdp_topics(hdp)

In [ ]:
hdp_topics[0]

[('believe', 0.002693449379876256),
 ('think', 0.00257707666605711),
 ('law', 0.002481670817360282),
 ('make', 0.0024207490496337414),
 ('religion', 0.0022249806206673384),
 ('may', 0.002082108287140727),
 ('see', 0.0020164246670901775),
 ('right', 0.0019723433069884777),
 ('many', 0.00193266780115664),
 ('state', 0.0019238201202824712)]

In [ ]:
import gensim.corpora as corpora
from gensim.models import CoherenceModel

# Build gensim objects
vocab = corpora.Dictionary(word_list_lemmatized)
corpus = [vocab.doc2bow(words) for words in word_list_lemmatized]

# Build topic list from dictionary
topic_list=[]
for k, tups in hdp_topics.items():
    topic_tokens=[]
    for w, p in tups:
        topic_tokens.append(w)

    topic_list.append(topic_tokens)

In [ ]:
topic_list[0]

['believe',
 'think',
 'law',
 'make',
 'religion',
 'may',
 'see',
 'right',
 'many',
 'state']

In [ ]:
cm = CoherenceModel(topics=topic_list, corpus=corpus, dictionary=vocab, texts=word_list_lemmatized,
                    coherence='c_v')

cm.get_coherence()

0.5963344200118257

In [ ]:
test_doc = word_list_lemmatized[0]

doc_inst = hdp.make_doc(test_doc)

topic_dist, ll = hdp.infer(doc_inst)

topic_idx = np.array(topic_dist).argmax()
topic_idx

3

In [ ]:
hdp.get_topic_words(topic_idx)

[('car', 0.006628578063100576),
 ('bike', 0.0040488168597221375),
 ('ride', 0.0026391283608973026),
 ('get', 0.0024984274059534073),
 ('engine', 0.0023395137395709753),
 ('drive', 0.002275686478242278),
 ('buy', 0.002237173495814204),
 ('good', 0.0021873542573302984),
 ('wire', 0.0021769863087683916),
 ('look', 0.0020851497538387775)]

In [ ]:
news_df.head(5)

,topic_id,content
0,7,From: lerxst@wam.umd.edu (where's my thing)\nS...
1,4,From: guykuo@carson.u.washington.edu (Guy Kuo)...
2,4,From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3,1,From: jgreen@amber (Joe Green)\nSubject: Re: W...
4,14,From: jcm@head-cfa.harvard.edu (Jonathan McDow...


In [ ]:
!pip install scripts

In [ ]:
from gensim.models import HdpModel

# Filter outlier words (e.g. appear <10 docs or in over 50% of docs)
vocab = corpora.Dictionary(word_list_lemmatized)
vocab.filter_extremes(no_below=20, no_above=0.5)
vocab.compactify()

# Build corpus
corpus = [vocab.doc2bow(words) for words in word_list_lemmatized]

# Just changing the learning rate and truncation level
hdp = HdpModel(corpus, vocab, gamma=1, T=30, alpha=0.1, K=8, kappa=1, random_state=20) # default kappa
hdp08 = HdpModel(corpus, vocab, gamma=1, T=30, alpha=0.1, K=8, kappa=0.8, random_state=20)
hdp06 = HdpModel(corpus, vocab, gamma=1, T=30, alpha=0.1, K=8, kappa=0.6, random_state=20)

In [ ]:
vb_model = [hdp06, hdp08, hdp]
vb_coherence =[]

for m in vb_model:
    cm = CoherenceModel(m, texts=word_list_lemmatized, corpus=corpus, dictionary=vocab, coherence='c_v')
    vb_coherence.append(cm.get_coherence())

In [ ]:
pd.DataFrame(vb_coherence, index=['hdp_kap06', 'hdp_kap08', 'hdp_kap1']).T

,hdp_kap06,hdp_kap08,hdp_kap1
0,0.328488,0.422874,0.499369
